In [1]:
import numpy as np
import healpy as hp
from astropy.table import Table

In [2]:
NSIDE = 64
npix = hp.nside2npix(NSIDE)
total_degrees_in_sky = 4.*np.pi*(180./np.pi)**2
area_per_pixel = total_degrees_in_sky/(1.*npix)

In [3]:
def radec_to_sph(ra,dc):
    
    theta = (90.-dc)*np.pi/180.
    phi   = ra*np.pi/180.
    return theta, phi

In [4]:
def get_n(ra,dec,NSIDE):
    #Convert to ra/dec
    theta, phi = radec_to_sph(ra,dec)
    #set up the healpix grid.
    n = hp.ang2pix(NSIDE, theta, phi)    
    return n

### Median F-test catalog density

In [5]:
#First, let's get the median density of the F-test selected sources. 
Fcat = Table.read("../v1.1/Master_Catalog_F_test_AGN.fits")
Fcat2 = Table.read("../v1.1_complementary/Master_Catalog_F_test_AGN.complementary.fits")

In [6]:
#Get the healpix pixel numbers.
n_Fcat  = get_n(Fcat['ra'] , Fcat['dec'] , NSIDE)
n_Fcat2 = get_n(Fcat2['ra'], Fcat2['dec'], NSIDE)
n_Ftot = np.concatenate([n_Fcat, n_Fcat2])

In [7]:
#Get the median field density
h_Ftot = np.histogram(n_Ftot,hp.nside2npix(NSIDE), range=(0,hp.nside2npix(NSIDE)-1))[0]
median_F_number = np.median(h_Ftot[h_Ftot>0])
median_F_density = median_F_number/area_per_pixel

In [8]:
print("The median density of F-test sources is {:.1f} per sq. deg.".format(median_F_density))
print("In regions that are not gaps or deeper fields, there are a total of {} sources".format(np.sum(h_Ftot[(h_Ftot>0.2*median_F_number) & (h_Ftot<3*median_F_number)])))

The median density of F-test sources is 97.7 per sq. deg.
In regions that are not gaps or deeper fields, there are a total of 1422505 sources


In [9]:
#Now, we need to define a gap density and an overdense field density to make the final catalog. To define the gap regions, we need to get the BIC densities. 
BIC_cat = Table.read("../v1.1/Master_Catalog_BIC_AGN_nb_le_6.fits")
BIC_cat2 = Table.read("../v1.1_complementary/Master_Catalog_BIC_AGN_nb_le_6.complementary.fits")

In [10]:
n_BIC_cat  = get_n(BIC_cat['ra'] , BIC_cat['dec'] , NSIDE)
n_BIC_cat2 = get_n(BIC_cat2['ra'], BIC_cat2['dec'], NSIDE)
n_BIC = np.concatenate([n_BIC_cat, n_BIC_cat2])

In [11]:
h_BIC = np.histogram(n_BIC,hp.nside2npix(NSIDE), range=(0,hp.nside2npix(NSIDE)-1))[0]

### Estimate the number of sources

In [12]:
#Define the main survey, gaps and high density regions. 
main_survey_condition = (h_Ftot>=0.2*median_F_number) & (h_Ftot<=3*median_F_number)
gap_condition = (h_BIC>0) & (h_Ftot<0.2*median_F_number)
high_density_condition = h_Ftot>3*median_F_number

#Now, get the total number of sources that the final catalog will have if we keep the median number in all the cells of the main F-test survey region, half of that in the gaps and twice that in the high density regions. 
N_FR = len(h_Ftot[main_survey_condition])*median_F_number
N_GR = len(h_Ftot[gap_condition])*median_F_number*0.5
N_HR = len(h_Ftot[high_density_condition])*median_F_number*2
print(N_FR, N_GR, N_HR, N_FR+N_GR+N_HR)

1237790.0 240752.0 277324.0 1755866.0


### Select the sources

In [13]:
#This is the histogram that will hold for each healpix pixel the number of sources in the final catalog. 
h_final = np.zeros(len(h_Ftot))

In [14]:
#We will create an extra column for each table where we will make notice of whether a given source makes it or not in the final table. 
Fcat['Final_cat']     = np.zeros(len(Fcat['ra']), dtype=np.int32)
Fcat2['Final_cat']    = np.zeros(len(Fcat2['ra']), dtype=np.int32)
BIC_cat['Final_cat']  = np.zeros(len(BIC_cat['ra']), dtype=np.int32)
BIC_cat2['Final_cat'] = np.zeros(len(BIC_cat2['ra']), dtype=np.int32)

In [16]:
for n in range(npix):

    #If there are no sources, skip to the next step. 
    if h_BIC[n]==0:
        continue

    #Set the target number of sources. 
    targ_num = median_F_number
    if gap_condition[n]:
        targ_num = 0.5*median_F_number
    elif high_density_condition[n]:
        targ_num = 2.0*median_F_number
    targ_num = np.int32(targ_num)

    #Make slice conditions. 
    cond_F1 = n_Fcat==n
    cond_F2 = n_Fcat2==n
    cond_B1 = n_BIC_cat==n
    cond_B2 = n_BIC_cat2==n

    #Get the number of F-test sources and BIC sources in each pixel. 
    F_num = len(Fcat['ra'][cond_F1]) + len(Fcat2['ra'][cond_F2])
    B_num = len(BIC_cat['ra'][cond_B1]) + len(BIC_cat2['ra'][cond_B2])

    #If we have enough F-test sources, just use those and keep only the ones with the lowest probability. 
    if F_num>=targ_num:
        Fp_lim = np.sort(np.concatenate([Fcat['Fp'][cond_F1], Fcat2['Fp'][cond_F2]]))[targ_num-1]
        Fcat['Final_cat'][(cond_F1) & (Fcat['Fp']<=Fp_lim)] = 1
        Fcat2['Final_cat'][(cond_F2) & (Fcat2['Fp']<=Fp_lim)] = 1

    #If we have enough sources between BIC and F-test, then use all the F-test sources and the best BIC sources. 
    elif F_num + B_num > targ_num:
        Fcat['Final_cat'][cond_F1] = 1
        Fcat2['Final_cat'][cond_F2] = 1
        aux = np.sort(np.concatenate([BIC_cat['BIC'][cond_B1], BIC_cat2['BIC'][cond_B2]]))[::-1]
        BIC_lim = aux[targ_num-F_num]
        BIC_cat['Final_cat'][(cond_B1) & (BIC_cat['BIC']>=BIC_lim)] = 1
        BIC_cat2['Final_cat'][(cond_B2) & (BIC_cat2['BIC']>=BIC_lim)] = 1

    #Finally, if not enough between the two of them, just use all sources. 
    else:
        Fcat['Final_cat'][cond_F1] = 1
        Fcat2['Final_cat'][cond_F2] = 1
        BIC_cat['Final_cat'][cond_B1] = 1
        BIC_cat2['Final_cat'][cond_B2] = 1

In [ ]:
#Save the catalogs.
Fcat_save = Fcat[Fcat['Final_cat']==1]
Fcat_save.write("Master_Catalog_F_test_AGN.fits")

Fcat2_save = Fcat2[Fcat2['Final_cat']==1]
Fcat2_save.write("Master_Catalog_F_test_AGN.complementary.fits")

BIC_cat_save = BIC_cat[BIC_cat['Final_cat']==1]
BIC_cat_save.write("Master_Catalog_BIC_AGN_nb_le_6.fits")

BIC_cat2_save = BIC_cat2[BIC_cat2['Final_cat']==1]
BIC_cat2_save.write("Master_Catalog_BIC_AGN_nb_le_6.complementary.fits")